### Добудем данные

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("joebeachcapital/airbnb")

print("Path to dataset files:", path)

100%|██████████| 1.01G/1.01G [00:31<00:00, 34.4MB/s]

Extracting model files...


Path to dataset files: /home/ubuntu/.cache/kagglehub/datasets/joebeachcapital/airbnb/versions/1


In [4]:
!cp -r /home/ubuntu/.cache/kagglehub/datasets/joebeachcapital/airbnb/versions/1 /home/ubuntu

In [6]:
!ls 1

airbnb-listings.csv  airbnb-listings.geojson


In [7]:
!hdfs dfs -put 1/airbnb-listings.csv /user/airbnb

In [8]:
!hdfs dfs -ls /user/airbnb

Found 1 items
-rw-r--r--   1 ubuntu hadoop 1935924731 2025-03-27 19:44 /user/airbnb/airbnb-listings.csv


### Достаем из карманов спарк

In [9]:
import findspark
findspark.init()

In [10]:
from pyspark.sql import SparkSession
import pyspark

sc = pyspark.SparkContext(appName="bonus")
spark = SparkSession(sc)

In [15]:
data = spark.read.option("mode", "DROPMALFORMED").option('sep', ';') \
.csv("/user/airbnb/airbnb-listings.csv", header=True, inferSchema=True)

In [16]:
data.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Listing Url: string (nullable = true)
 |-- Scrape ID: string (nullable = true)
 |-- Last Scraped: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Space: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Experiences Offered: string (nullable = true)
 |-- Neighborhood Overview: string (nullable = true)
 |-- Notes: string (nullable = true)
 |-- Transit: string (nullable = true)
 |-- Access: string (nullable = true)
 |-- Interaction: string (nullable = true)
 |-- House Rules: string (nullable = true)
 |-- Thumbnail Url: string (nullable = true)
 |-- Medium Url: string (nullable = true)
 |-- Picture Url: string (nullable = true)
 |-- XL Picture Url: string (nullable = true)
 |-- Host ID: string (nullable = true)
 |-- Host URL: string (nullable = true)
 |-- Host Name: string (nullable = true)
 |-- Host Since: string (nullable = true)
 |-- Host Location: string (nullable

In [17]:
data.limit(4).toPandas()

,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,...,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features
0,8264596,https://www.airbnb.com/rooms/8264596,20170402075052,2017-04-02,Cosy apartment across Vondelpark,This central located apartment will make you f...,None,This central located apartment will make you f...,none,None,...,10,10,10,None,Amsterdam,flexible,1,0.05,"52.361943953804776, 4.866686522141489","Host Has Profile Pic,Host Identity Verified,Is..."
1,2180729,https://www.airbnb.com/rooms/2180729,20170402075052,2017-04-02,Spacious City Apartment Oud-West,Living like an Amsterdam resident in this ligh...,There are 2 bedrooms each with a double bed. I...,Living like an Amsterdam resident in this ligh...,none,The 85 m2 apartment is located on a quiet stre...,...,9,9,9,None,Amsterdam,flexible,1,0.62,"52.37014616115532, 4.866281767968382","Host Has Profile Pic,Host Identity Verified,Is..."
2,14463171,https://www.airbnb.com/rooms/14463171,20170402075052,2017-04-02,Cosy Studio Apartment Center Amsterdam,Studio Apartment Centre Amsterdam Perfect stud...,None,Studio Apartment Centre Amsterdam Perfect stud...,none,None,...,8,9,9,None,Amsterdam,moderate,1,2.04,"52.368817829014425, 4.871249476925455",Host Has Profile Pic
3,13932749,https://www.airbnb.com/rooms/13932749,20170402075052,2017-04-02,Spacious apt with sunny balcony,This beautiful 80m2 apartment has a huge livin...,"The huge living room and open kitchen, make th...",This beautiful 80m2 apartment has a huge livin...,none,"Amsterdam West is a colorful, bustling and up...",...,10,9,10,None,Amsterdam,strict,1,1.49,"52.36949764400226, 4.859321434670662","Host Is Superhost,Host Has Profile Pic,Host Id..."


### Теперь к заданиям

Задание 1. [1 балл] В разрезе каждого города (колонка city) выделите весь список удобств (amenities), предлагаемых в жилплощади из этих городов (учитывайте возможные дубли в этих данных). В ответе выведите фрейм данных со списком всех удобств для следующих городов: Hoxton, Rainham, Rio Del Mar. По ответам на эти города мы увидим, правильные ли преобразования для данных вы собрали.

In [21]:
data.select("city", "amenities").show(10)

+---------+--------------------+
|     city|           amenities|
+---------+--------------------+
|     null|                null|
|     null|                null|
|     null|                null|
|       50|                   9|
|     null|                null|
|     null|                null|
|     null|                null|
|       30|                   9|
|Amsterdam|TV,Cable TV,Inter...|
|Amsterdam|TV,Cable TV,Inter...|
+---------+--------------------+
only showing top 10 rows



In [24]:
from pyspark.sql.functions import explode, split, col, collect_set

distincted_data = data.select("City", explode(split(col("Amenities"), ','))).groupBy("City").agg(collect_set("col"))
filtered_data = distincted_data.filter(col("City").isin("Hoxton", "Rainham", "Rio Del Mar"))
filtered_data.show(truncate=False)

+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|City       |collect_set(col)                                                                                                                                                    |
+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Rainham    |[Wireless Internet, Essentials, Iron, Washer, TV, Heating, Kitchen, translation missing: en.hosting_amenity_49, translation missing: en.hosting_amenity_50, Hangers]|
|Hoxton     |[Wireless Internet, Essentials, Iron, TV, Heating, Smoke detector, Internet, Laptop friendly workspace, Kitchen, Hangers]                                           |
|Rio Del Mar|[Indoor Fireplace, Suitable for Events, Essentials, Shampoo, Washer, Heating, Kitchen, Free 

Задание 2. [1 балл] Также в разрезе каждого города выделите удобства, которые предлагаются в жилплощадях этих городов и выделите из них топ-5 городов, в жилье которых предлагается больше всего удобств. Учитывайте дубли в этих значениях относительно разного жилья, тут речь идет про поиск всех уникальных значений по городам и их подсчет для поиска самых удобных городов. В качестве ответа выведите фрейм данных с этими городами и числом уникальных удобств из них.

In [25]:
from pyspark.sql.functions import explode, split, col, countDistinct

amenities_count = (
    data.select("City", explode(split(col("Amenities"), ','))
         .alias("Amenity"))
    .groupBy("City")
    .agg(countDistinct("Amenity").alias("Unique_Amenities_Count"))
)
top_5_cities = amenities_count.orderBy(col("Unique_Amenities_Count").desc()).limit(5)
top_5_cities.show(truncate=False)

+----------+----------------------+
|City      |Unique_Amenities_Count|
+----------+----------------------+
|null      |136                   |
|Toronto   |108                   |
|København |105                   |
|Roma      |104                   |
|Washington|102                   |
+----------+----------------------+



Задание 3. [1 балл] Давайте выясним, по каким объявлениям можно пройти верификацию на оффер по жилью через linkedin (колонка host_verifications), что в реалиях airbnb достаточно забавно. Нам также подойдут варианты объявлений, где верификация через линкед будет одним из возможных вариантов. В ответе укажите кол-во объявлений из стран (колонка country) Switzerland и Austria, по которым можно пройти верификацию через линкед.

In [26]:
from pyspark.sql.functions import col, lower

linkedin_verifications = data.filter(
    (lower(col("Host Verifications")).like("%linkedin%")) &
    (col("Country").isin("Switzerland", "Austria"))
)
result = linkedin_verifications.groupBy("Country").count()
result.show(truncate=False)


+-----------+-----+
|Country    |count|
+-----------+-----+
|Switzerland|9    |
|Austria    |70   |
+-----------+-----+



дальше не успел